In [0]:
IS_SPARK_SUBMIT_CLI = False
if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

In [0]:
spark.sparkContext.setLogLevel("WARN")

In [0]:
# File location and type
file_location = "/FileStore/tables/USA_USED_CARS_DATASET_FOR_PREDICTING_CAR_PRICE.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

#display(df)
df.show()

+-----------------+------------+----+----------+----------+---------------+-----+---------------+-----------------+--------------------+------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-

In [0]:

temp_table_name = "USA_USED_CARS_DATASET_FOR_PREDICTING_CAR_PRICE_csv"

df.createOrReplaceTempView(temp_table_name)

In [0]:
if IS_SPARK_SUBMIT_CLI:
   df = spark.read.csv('USA_USED_CARS_DATASET_FOR_PREDICTING_CAR_PRICE.csv', inferSchema=True, header=True)
else:
    df = spark.sql("SELECT * FROM USA_USED_CARS_DATASET_FOR_PREDICTING_CAR_PRICE_csv")


df.show(5)

+-----------------+------------+----+----------+----------+---------------+-----+------------+-----------------+--------------------+------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+----------------+--------------------+------------------+--------------------+------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+-------

In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from pyspark.sql.types import StructType, StructField, StringType,IntegerType
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.regression import RandomForestRegressor, LinearRegression, GBTRegressor, FMRegressor
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder,TrainValidationSplit, CrossValidator
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.classification import DecisionTreeClassifier, LogisticRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import when, col

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder
import pandas as pd



In [0]:
df2 = df.withColumn("year",col("year").cast("int")).withColumn("city_fuel_economy",col("city_fuel_economy").cast("float")).withColumn("price",col("price").cast("float")).withColumn("mileage",col("mileage").cast("float")).withColumn("owner_count",col("owner_count").cast("int")).withColumn("latitude",col("latitude").cast("float")).withColumn("highway_fuel_economy",col("highway_fuel_economy").cast("float")).withColumn("daysonmarket",col("daysonmarket").cast("int")).withColumn("dealer_zip",col("dealer_zip").cast("int")).withColumn("engine_displacement",col("engine_displacement").cast("int")).withColumn("listing_id",col("listing_id").cast("int")).withColumn("savings_amount",col("savings_amount").cast("int")).withColumn("sp_id",col("sp_id").cast("int")).withColumn("seller_rating",col("seller_rating").cast("int"))


In [0]:
(df2.select("year", "city_fuel_economy", "price", "mileage", "owner_count", "latitude", "highway_fuel_economy", "daysonmarket", "dealer_zip", "engine_displacement", "listing_id", "savings_amount", "seller_rating", "sp_id","frame_damaged","theft_title", "has_accidents", "frame_damaged", "theft_title").limit(10)).show()

+----+-----------------+-------+--------+-----------+--------+--------------------+------------+----------+-------------------+----------+--------------+-------------+------+--------------------+------------------+--------------------+--------------------+------------------+
|year|city_fuel_economy|  price| mileage|owner_count|latitude|highway_fuel_economy|daysonmarket|dealer_zip|engine_displacement|listing_id|savings_amount|seller_rating| sp_id|       frame_damaged|       theft_title|       has_accidents|       frame_damaged|       theft_title|
+----+-----------------+-------+--------+-----------+--------+--------------------+------------+----------+-------------------+----------+--------------+-------------+------+--------------------+------------------+--------------------+--------------------+------------------+
|2019|             19.0|55994.0|    14.0|       null|  30.222|                26.0|         716|     32244|               3500| 219569238|             0|            4|37178

In [0]:
df3 = df2.toPandas()

In [0]:
data1=df2

In [0]:
df2.na.drop(subset=["price", "year", "city_fuel_economy", "mileage", "owner_count", "highway_fuel_economy", "daysonmarket", "dealer_zip", "engine_displacement", "listing_id", "savings_amount", "seller_rating", "sp_id", "has_accidents", "frame_damaged", "theft_title"])
df222 = df2.filter(col("price") < 100000)
df22 = df222.select("year", "city_fuel_economy", col("price").alias("label"), "mileage", "owner_count", "latitude", "highway_fuel_economy", "daysonmarket", "dealer_zip", "engine_displacement", "listing_id", "savings_amount", "seller_rating", "sp_id", "has_accidents", "frame_damaged", "theft_title")
df22.show()



+----+-----------------+-------+--------+-----------+--------+--------------------+------------+----------+-------------------+----------+--------------+-------------+------+-------------+-------------+-----------+
|year|city_fuel_economy|  label| mileage|owner_count|latitude|highway_fuel_economy|daysonmarket|dealer_zip|engine_displacement|listing_id|savings_amount|seller_rating| sp_id|has_accidents|frame_damaged|theft_title|
+----+-----------------+-------+--------+-----------+--------+--------------------+------------+----------+-------------------+----------+--------------+-------------+------+-------------+-------------+-----------+
|2019|             19.0|55994.0|    14.0|       null|  30.222|                26.0|         716|     32244|               3500| 219569238|             0|            4|371785|         null|         null|       null|
|2018|             22.0|35900.0| 19559.0|          1| 30.2869|                28.0|         100|     32246|               2000| 273093755|  

In [0]:
df22.show()

+----+-----------------+-------+--------+-----------+--------+--------------------+------------+----------+-------------------+----------+--------------+-------------+------+-------------+-------------+-----------+
|year|city_fuel_economy|  label| mileage|owner_count|latitude|highway_fuel_economy|daysonmarket|dealer_zip|engine_displacement|listing_id|savings_amount|seller_rating| sp_id|has_accidents|frame_damaged|theft_title|
+----+-----------------+-------+--------+-----------+--------+--------------------+------------+----------+-------------------+----------+--------------+-------------+------+-------------+-------------+-----------+
|2019|             19.0|55994.0|    14.0|       null|  30.222|                26.0|         716|     32244|               3500| 219569238|             0|            4|371785|         null|         null|       null|
|2018|             22.0|35900.0| 19559.0|          1| 30.2869|                28.0|         100|     32246|               2000| 273093755|  

In [0]:
splits = df22.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("TRAINING ROWS:", train_rows, "TESTING ROWS:", test_rows)

TRAINING ROWS: 2966 TESTING ROWS: 1323


In [0]:
xlr_assembler = VectorAssembler(inputCols = ["year", "city_fuel_economy", "mileage", "owner_count", "highway_fuel_economy", "engine_displacement", "savings_amount"], outputCol="features").setHandleInvalid("skip")

In [0]:
lr= LinearRegression(labelCol="label") #,featuresCol="features",maxIter=10

In [0]:
paramGrid1_CV_LR = ParamGridBuilder()\
.addGrid(lr.maxIter, [10, 30])\
.addGrid(lr.regParam, [0, 0.1])\
.build()

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
lr_CV_evaluator = RegressionEvaluator(predictionCol="prediction", \
labelCol="label",metricName="r2")


In [0]:
#we can use same pipeline_lr for both cv and tvs
pipeline_lr = Pipeline(stages=[lr_assembler, lr]) 

In [0]:
cv_lr = CrossValidator(estimator=pipeline_lr, evaluator=lr_CV_evaluator, estimatorParamMaps=paramGrid1_CV_LR)

In [0]:
model_CV_LR = cv_lr.fit(train) 

/databricks/spark/python/pyspark/ml/util.py:886: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
  warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)


In [0]:
prediction_CV_lR = model_CV_LR.transform(test)
predicted_CV_lR = prediction_CV_lR.select("features", "prediction", "label") 
predicted_CV_lR.show(10)

+--------------------+------------------+-------+
|            features|        prediction|  label|
+--------------------+------------------+-------+
|[1991.0,13.0,7583...| 4815.534744219389| 8995.0|
|[1994.0,15.0,6960...| 6527.798986880574| 9999.0|
|[2002.0,14.0,1712...|  8457.52107790159| 8950.0|
|[2004.0,16.0,1434...| 2238.519381177146| 4889.0|
|[2006.0,13.0,1483...|15546.763351364061| 7489.0|
|[2006.0,17.0,1276...| 5836.667753734626| 4488.0|
|[2006.0,17.0,1365...| 5872.726446398534| 5589.0|
|[2007.0,12.0,1574...|15185.395905680954|11850.0|
|[2007.0,16.0,6251...|32209.607766159344|34998.0|
|[2011.0,16.0,7325...|  19525.6638359949|11989.0|
+--------------------+------------------+-------+
only showing top 10 rows



In [0]:
#display(predicted_lr.limit(1000))
(predicted_CV_lR.limit(1000)).show()

+--------------------+------------------+-------+
|            features|        prediction|  label|
+--------------------+------------------+-------+
|[1991.0,13.0,7583...| 4815.534744219389| 8995.0|
|[1994.0,15.0,6960...| 6527.798986880574| 9999.0|
|[2002.0,14.0,1712...|  8457.52107790159| 8950.0|
|[2004.0,16.0,1434...| 2238.519381177146| 4889.0|
|[2006.0,13.0,1483...|15546.763351364061| 7489.0|
|[2006.0,17.0,1276...| 5836.667753734626| 4488.0|
|[2006.0,17.0,1365...| 5872.726446398534| 5589.0|
|[2007.0,12.0,1574...|15185.395905680954|11850.0|
|[2007.0,16.0,6251...|32209.607766159344|34998.0|
|[2011.0,16.0,7325...|  19525.6638359949|11989.0|
|[2011.0,18.0,8840...| 17522.29364937218|12921.0|
|[2011.0,23.0,8090...| 10914.43958372064| 7789.0|
|[2012.0,14.0,9504...|  28669.6184711135|19391.0|
|[2012.0,15.0,1242...| 23691.79728942318|16389.0|
|[2012.0,18.0,1224...|15315.745126764756|10789.0|
|[2012.0,25.0,1147...|  5244.58752431022| 9489.0|
|[2013.0,17.0,1042...|20024.063258262817|13604.0|


In [0]:
#print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(prediction_lr))

from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator_CV_r2 = RegressionEvaluator(predictionCol="prediction", \
                labelCol="label",metricName="r2")
 
lr_evaluator_CV_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse") 
#R2 and RMSE For LR_CV
print("r2 For LR_CV: %.2f" % lr_evaluator_CV_r2.evaluate(prediction_CV_lR))
print("RMSE For LR_CV: %.2f" % lr_evaluator_CV_rmse.evaluate(prediction_CV_lR))

r2 For LR_CV: 0.61
RMSE For LR_CV: 7693.73


In [0]:
paramGrid2_TV_LR = ParamGridBuilder()\
.addGrid(lr.maxIter, [10, 40])\
.addGrid(lr.regParam, [0.1, 1.0])\
.build()

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
lr_tv_evaluator = RegressionEvaluator(predictionCol="prediction", \
labelCol="label",metricName="r2")

In [0]:
tv_lr = TrainValidationSplit(estimator=pipeline_lr, evaluator=lr_tv_evaluator, estimatorParamMaps=paramGrid2_TV_LR, trainRatio=0.8)


In [0]:
model1_tv_lr = tv_lr.fit(train)

In [0]:
prediction_tv_lr = model1_tv_lr.transform(test)
predicted_tv_lr = prediction_tv_lr.select("features", "prediction", "label") 
predicted_tv_lr.show(10)

+--------------------+------------------+-------+
|            features|        prediction|  label|
+--------------------+------------------+-------+
|[1991.0,13.0,7583...| 4815.534744219389| 8995.0|
|[1994.0,15.0,6960...| 6527.798986880574| 9999.0|
|[2002.0,14.0,1712...|  8457.52107790159| 8950.0|
|[2004.0,16.0,1434...| 2238.519381177146| 4889.0|
|[2006.0,13.0,1483...|15546.763351364061| 7489.0|
|[2006.0,17.0,1276...| 5836.667753734626| 4488.0|
|[2006.0,17.0,1365...| 5872.726446398534| 5589.0|
|[2007.0,12.0,1574...|15185.395905680954|11850.0|
|[2007.0,16.0,6251...|32209.607766159344|34998.0|
|[2011.0,16.0,7325...|  19525.6638359949|11989.0|
+--------------------+------------------+-------+
only showing top 10 rows



In [0]:
#display(predicted_tv_lr.limit(1000))
(predicted_tv_lr.limit(1000)).show()

+--------------------+------------------+-------+
|            features|        prediction|  label|
+--------------------+------------------+-------+
|[1991.0,13.0,7583...| 4815.534744219389| 8995.0|
|[1994.0,15.0,6960...| 6527.798986880574| 9999.0|
|[2002.0,14.0,1712...|  8457.52107790159| 8950.0|
|[2004.0,16.0,1434...| 2238.519381177146| 4889.0|
|[2006.0,13.0,1483...|15546.763351364061| 7489.0|
|[2006.0,17.0,1276...| 5836.667753734626| 4488.0|
|[2006.0,17.0,1365...| 5872.726446398534| 5589.0|
|[2007.0,12.0,1574...|15185.395905680954|11850.0|
|[2007.0,16.0,6251...|32209.607766159344|34998.0|
|[2011.0,16.0,7325...|  19525.6638359949|11989.0|
|[2011.0,18.0,8840...| 17522.29364937218|12921.0|
|[2011.0,23.0,8090...| 10914.43958372064| 7789.0|
|[2012.0,14.0,9504...|  28669.6184711135|19391.0|
|[2012.0,15.0,1242...| 23691.79728942318|16389.0|
|[2012.0,18.0,1224...|15315.745126764756|10789.0|
|[2012.0,25.0,1147...|  5244.58752431022| 9489.0|
|[2013.0,17.0,1042...|20024.063258262817|13604.0|


In [0]:

from pyspark.ml.evaluation import RegressionEvaluator
lr_tv_evaluator_r2 = RegressionEvaluator(predictionCol="prediction", \
                labelCol="label",metricName="r2")
 
lr_tv_evaluator_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse") 
#R2 and RMSE For LR_TV
print("r2 For LR CV: %.2f" % lr_tv_evaluator_r2.evaluate(prediction_tv_lr))
print("RMSE for LT TV: %.2f" % lr_tv_evaluator_rmse.evaluate(prediction_tv_lr))


r2 For LR CV: 0.61
RMSE for LT TV: 7693.73


In [0]:
gbt_assembler = VectorAssembler(inputCols = ["year", "city_fuel_economy", "mileage", "owner_count", "highway_fuel_economy", "engine_displacement", "savings_amount"], outputCol="features").setHandleInvalid("skip")

In [0]:
gbt = GBTRegressor(labelCol="label") #,featuresCol="features",maxIter=10

In [0]:
paramGrid_CV_GBT = ParamGridBuilder()\
.addGrid(gbt.maxDepth, [2, 5])\
.addGrid(gbt.maxIter, [10, 20])\
.addGrid(gbt.minInfoGain, [0,0])\
.build()

In [0]:
gbt_evaluator = RegressionEvaluator(predictionCol="prediction", \
labelCol="label",metricName="r2")

In [0]:
pipeline_GBT = Pipeline(stages=[gbt_assembler, gbt]) 

In [0]:
GBT_CV = CrossValidator(estimator=pipeline_GBT, evaluator=gbt_evaluator, estimatorParamMaps=paramGrid_CV_GBT, numFolds = 3)

In [0]:
model_GBT = GBT_CV.fit(train) 

In [0]:
prediction_GBT = model_GBT.transform(test)
predicted_GBT = prediction_GBT.select("features", "prediction", "label") 
predicted_GBT.show(10)

+--------------------+------------------+-------+
|            features|        prediction|  label|
+--------------------+------------------+-------+
|[1991.0,13.0,7583...| 7802.411208415451| 8995.0|
|[1994.0,15.0,6960...| 9705.584285513498| 9999.0|
|[2002.0,14.0,1712...|12229.696964204248| 8950.0|
|[2004.0,16.0,1434...| 5727.033835482392| 4889.0|
|[2006.0,13.0,1483...| 6224.394710595671| 7489.0|
|[2006.0,17.0,1276...| 6217.972778736959| 4488.0|
|[2006.0,17.0,1365...| 6342.372046874887| 5589.0|
|[2007.0,12.0,1574...| 8311.900035425657|11850.0|
|[2007.0,16.0,6251...| 11033.35257580522|34998.0|
|[2011.0,16.0,7325...|  9456.82107384449|11989.0|
+--------------------+------------------+-------+
only showing top 10 rows



In [0]:
#display(predicted.limit(1000))
(predicted_GBT.limit(1000)).show()

+--------------------+------------------+-------+
|            features|        prediction|  label|
+--------------------+------------------+-------+
|[1991.0,13.0,7583...| 7802.411208415451| 8995.0|
|[1994.0,15.0,6960...| 9705.584285513498| 9999.0|
|[2002.0,14.0,1712...|12229.696964204248| 8950.0|
|[2004.0,16.0,1434...| 5727.033835482392| 4889.0|
|[2006.0,13.0,1483...| 6224.394710595671| 7489.0|
|[2006.0,17.0,1276...| 6217.972778736959| 4488.0|
|[2006.0,17.0,1365...| 6342.372046874887| 5589.0|
|[2007.0,12.0,1574...| 8311.900035425657|11850.0|
|[2007.0,16.0,6251...| 11033.35257580522|34998.0|
|[2011.0,16.0,7325...|  9456.82107384449|11989.0|
|[2011.0,18.0,8840...|  9426.67060123703|12921.0|
|[2011.0,23.0,8090...| 9993.335382008723| 7789.0|
|[2012.0,14.0,9504...|15358.909084474635|19391.0|
|[2012.0,15.0,1242...|14270.789479348681|16389.0|
|[2012.0,18.0,1224...|  8772.09949988705|10789.0|
|[2012.0,25.0,1147...| 8965.686976683617| 9489.0|
|[2013.0,17.0,1042...|14846.190823081979|13604.0|


In [0]:
gbt_evaluator_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse") 
print("RMSE_GBT_CV: %.2f" % gbt_evaluator_r2.evaluate(prediction_GBT))

gbt_evaluator_rmse = RegressionEvaluator(predictionCol="prediction", \
                labelCol="label",metricName="r2")
print("R2_GBT_CV: %.2f" % gbt_evaluator_rmse.evaluate(prediction_GBT))


RMSE_GBT_CV: 7059.37
R2_GBT_CV: 0.67


In [0]:
paramGrid_TV_GBT = ParamGridBuilder()\
.addGrid(gbt.maxDepth, [2, 5])\
.addGrid(gbt.maxIter, [20, 40])\
.addGrid(gbt.maxBins, [15, 30])\
.addGrid(gbt.stepSize, [0.05, 0.1])\
.addGrid(gbt.minInfoGain, [0,0])\
.build()

In [0]:
GBT_TV = TrainValidationSplit(estimator=pipeline_GBT, evaluator=gbt_evaluator, estimatorParamMaps=paramGrid_TV_GBT, trainRatio=0.8)


In [0]:
model_GBT_TV = GBT_TV.fit(train)

In [0]:
prediction_GBT = model_GBT_TV.transform(test)
predicted_GBT = prediction_GBT.select("features", "prediction", "label") 
predicted_GBT.show(10)

+--------------------+------------------+-------+
|            features|        prediction|  label|
+--------------------+------------------+-------+
|[1991.0,13.0,7583...| 6939.270627278039| 8995.0|
|[1994.0,15.0,6960...| 8592.925995893613| 9999.0|
|[2002.0,14.0,1712...|31973.636533407356| 8950.0|
|[2004.0,16.0,1434...| 3690.563577434275| 4889.0|
|[2006.0,13.0,1483...| 5799.857631250606| 7489.0|
|[2006.0,17.0,1276...| 5410.587937303878| 4488.0|
|[2006.0,17.0,1365...| 4786.050468158109| 5589.0|
|[2007.0,12.0,1574...| 9136.380471439339|11850.0|
|[2007.0,16.0,6251...| 72125.95353113592|34998.0|
|[2011.0,16.0,7325...|11526.751620197414|11989.0|
+--------------------+------------------+-------+
only showing top 10 rows



In [0]:
#display(predicted.limit(1000))
predicted_GBT.limit(1000).show()

+--------------------+------------------+-------+
|            features|        prediction|  label|
+--------------------+------------------+-------+
|[1991.0,13.0,7583...| 6939.270627278039| 8995.0|
|[1994.0,15.0,6960...| 8592.925995893613| 9999.0|
|[2002.0,14.0,1712...|31973.636533407356| 8950.0|
|[2004.0,16.0,1434...| 3690.563577434275| 4889.0|
|[2006.0,13.0,1483...| 5799.857631250606| 7489.0|
|[2006.0,17.0,1276...| 5410.587937303878| 4488.0|
|[2006.0,17.0,1365...| 4786.050468158109| 5589.0|
|[2007.0,12.0,1574...| 9136.380471439339|11850.0|
|[2007.0,16.0,6251...| 72125.95353113592|34998.0|
|[2011.0,16.0,7325...|11526.751620197414|11989.0|
|[2011.0,18.0,8840...| 10156.73943801799|12921.0|
|[2011.0,23.0,8090...|10143.772425435795| 7789.0|
|[2012.0,14.0,9504...| 18116.06458289002|19391.0|
|[2012.0,15.0,1242...|14106.753554736548|16389.0|
|[2012.0,18.0,1224...|  9953.57517671495|10789.0|
|[2012.0,25.0,1147...| 8614.192872005264| 9489.0|
|[2013.0,17.0,1042...|13628.241252442709|13604.0|


In [0]:
gbt_evaluator_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse") 
print("RMSE_GBT_TV: %.2f" % gbt_evaluator_rmse.evaluate(prediction_GBT))

 
gbt_evaluator_r2 = RegressionEvaluator(predictionCol="prediction", \
                labelCol="label",metricName="r2")
print("R2_GBT_TV: %.2f" % gbt_evaluator_r2.evaluate(prediction_GBT))


RMSE_GBT_TV: 7009.82
R2_GBT_TV: 0.67


In [0]:
assembler_Rf = VectorAssembler(inputCols = ["year", "city_fuel_economy", "mileage", "owner_count", "highway_fuel_economy", "engine_displacement", "savings_amount"], outputCol="features").setHandleInvalid("skip")

In [0]:
training_Rf = assembler_Rf.transform(train)
#training.show()

In [0]:
rf = RandomForestRegressor(labelCol='label') #,featuresCol="features", numTrees=10, maxDepth=5

In [0]:
paramGrid_CV_Rf = ParamGridBuilder()\
    .addGrid(rf.maxDepth, [5, 10, 15])\
    .addGrid(rf.numTrees, [20, 50, 100])\
    .addGrid(rf.minInfoGain, [0.0])\
    .build()


In [0]:
rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
labelCol="label",metricName="r2")

In [0]:
pipeline_rf = Pipeline(stages=[assembler_Rf, rf])

In [0]:
cv_rf = CrossValidator(estimator=pipeline_rf, evaluator=rf_evaluator, estimatorParamMaps=paramGrid_CV_Rf)

In [0]:
model_CV_rf = cv_rf.fit(train)

In [0]:
prediction_CV_rf = model_CV_rf.transform(test)
predicted_CV_rf = prediction_CV_rf.select("features", "prediction", "label") 
predicted_CV_rf.show(10)

+--------------------+------------------+-------+
|            features|        prediction|  label|
+--------------------+------------------+-------+
|[1991.0,13.0,7583...|          10713.51| 8995.0|
|[1994.0,15.0,6960...|          12771.38| 9999.0|
|[2002.0,14.0,1712...| 9106.353333333334| 8950.0|
|[2004.0,16.0,1434...| 5547.738690476191| 4889.0|
|[2006.0,13.0,1483...| 7960.288333333333| 7489.0|
|[2006.0,17.0,1276...| 6426.022777777777| 4488.0|
|[2006.0,17.0,1365...|6088.0498611111125| 5589.0|
|[2007.0,12.0,1574...| 8685.243333333334|11850.0|
|[2007.0,16.0,6251...|40237.435333333335|34998.0|
|[2011.0,16.0,7325...|15776.226499999999|11989.0|
+--------------------+------------------+-------+
only showing top 10 rows



In [0]:
#display(prediction_rf.limit(1000))
prediction_CV_rf.limit(1000).show()

+----+-----------------+-------+--------+-----------+--------+--------------------+------------+----------+-------------------+----------+--------------+-------------+------+-------------+-------------+-----------+--------------------+------------------+
|year|city_fuel_economy|  label| mileage|owner_count|latitude|highway_fuel_economy|daysonmarket|dealer_zip|engine_displacement|listing_id|savings_amount|seller_rating| sp_id|has_accidents|frame_damaged|theft_title|            features|        prediction|
+----+-----------------+-------+--------+-----------+--------+--------------------+------------+----------+-------------------+----------+--------------+-------------+------+-------------+-------------+-----------+--------------------+------------------+
|1991|             13.0| 8995.0| 75836.0|          3|  42.459|                17.0|          46|     50613|               5000| 277639817|             0|            4|284048|        FALSE|        FALSE|      FALSE|[1991.0,13.0,7583...|

In [0]:
rf_evaluator_CV_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse") 
print("RMSE for CV_RF: %.2f" % rf_evaluator_CV_rmse.evaluate(prediction_CV_rf))

 
rf_evaluator_CV_r2 = RegressionEvaluator(predictionCol="prediction", \
                labelCol="label",metricName="r2")
print("R2 for CV_RF: %.2f" % rf_evaluator_CV_r2.evaluate(prediction_CV_rf))

RMSE for CV_RF: 6197.13
R2 for CV_RF: 0.74


In [0]:
paramGrid_TV_Rf = ParamGridBuilder()\
    .addGrid(rf.maxDepth, [5, 10, 15])\
    .addGrid(rf.numTrees, [20, 50, 100])\
    .addGrid(rf.featureSubsetStrategy, ['auto', 'sqrt', 'log2'])\
    .addGrid(rf.minInfoGain, [0.0])\
    .build()


In [0]:
Tv_rf = TrainValidationSplit(estimator=pipeline_rf, evaluator=rf_evaluator, estimatorParamMaps=paramGrid_TV_Rf, trainRatio=0.9)


In [0]:
model_tv_rf = Tv_rf.fit(train)

/databricks/spark/python/pyspark/ml/util.py:886: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
  warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)


In [0]:
prediction_tv_rf = model_tv_rf.transform(test)
predicted_tv_rf = prediction_tv_rf.select("features", "prediction", "label") 
predicted_tv_rf.show(10)

+--------------------+------------------+-------+
|            features|        prediction|  label|
+--------------------+------------------+-------+
|[1997.0,17.0,1645...|           4841.15| 1500.0|
|[2002.0,14.0,1712...| 6577.741666666667| 8950.0|
|[2003.0,18.0,1482...|           3866.85| 2800.0|
|[2005.0,15.0,2820...|30949.783333333336|24900.0|
|[2006.0,15.0,1282...|            5755.7| 4995.0|
|[2006.0,17.0,1365...|           6358.25| 5589.0|
|[2006.0,20.0,8719...|7816.9857142857145| 5900.0|
|[2008.0,15.0,310....| 44642.00833333334|48999.0|
|[2008.0,17.0,1484...|          6421.875| 5495.0|
|[2008.0,22.0,1058...| 6423.666666666666| 5995.0|
+--------------------+------------------+-------+
only showing top 10 rows



In [0]:
#display(prediction_tv_rf.limit(1000))
prediction_tv_rf.limit(1000).show()

+----+-----------------+-------+--------+-----------+--------+--------------------+------------+----------+-------------------+----------+--------------+-------------+------+-------------+-------------+-----------+--------------------+------------------+
|year|city_fuel_economy|  label| mileage|owner_count|latitude|highway_fuel_economy|daysonmarket|dealer_zip|engine_displacement|listing_id|savings_amount|seller_rating| sp_id|has_accidents|frame_damaged|theft_title|            features|        prediction|
+----+-----------------+-------+--------+-----------+--------+--------------------+------------+----------+-------------------+----------+--------------+-------------+------+-------------+-------------+-----------+--------------------+------------------+
|1997|             17.0| 1500.0|164524.0|          2|  42.459|                27.0|          84|     50613|               3800| 274364588|             0|            4|284048|        FALSE|        FALSE|      FALSE|[1997.0,17.0,1645...|

In [0]:
rf_tv_evaluator_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse") 
print("RMSE for RF_TV: %.2f" % rf_tv_evaluator_rmse.evaluate(prediction_tv_rf))

 
rf_tv_evaluator_r2 = RegressionEvaluator(predictionCol="prediction", \
                labelCol="label",metricName="r2")
print("R2 for RF_TV: %.2f" % rf_tv_evaluator_r2.evaluate(prediction_tv_rf))

RMSE for RF_TV: 6758.30
R2 for RF_TV: 0.73


In [0]:
model_RF = pipeline_rf.fit(train)

In [0]:
rfModel = model_RF.stages[-1]
#print(rfModel.toDebugString)

In [0]:
featureImp = pd.DataFrame(list(zip(assembler_Rf.getInputCols(), rfModel.featureImportances)), columns=["feature", "importance"])
featureImp.sort_values(by="importance", ascending="False")

,feature,importance
3,owner_count,0.044117
6,savings_amount,0.065726
1,city_fuel_economy,0.092745
4,highway_fuel_economy,0.118275
5,engine_displacement,0.145099
0,year,0.230173
2,mileage,0.303864


In [0]:
rf = RandomForestRegressor(labelCol="label", featuresCol="features", numTrees=30, maxDepth=15)

# combine stages into pipeline
pipeline_RF_1 = Pipeline(stages= [assembler_Rf, rf])
model_rf = pipeline_RF_1.fit(train)

In [0]:
prediction_rf1 = model_rf.transform(test)
predicted_rf1 = prediction_rf1.select("features", "prediction", "label")
predicted_rf1.show(10)

+--------------------+------------------+-------+
|            features|        prediction|  label|
+--------------------+------------------+-------+
|[1997.0,17.0,1645...|3879.0555555555557| 1500.0|
|[2002.0,14.0,1712...|10613.944444444445| 8950.0|
|[2003.0,18.0,1482...|3778.5333333333333| 2800.0|
|[2005.0,15.0,2820...|31339.666666666668|24900.0|
|[2006.0,15.0,1282...| 6070.874358974358| 4995.0|
|[2006.0,17.0,1365...|5875.3076923076915| 5589.0|
|[2006.0,20.0,8719...|            7449.1| 5900.0|
|[2008.0,15.0,310....|           48372.8|48999.0|
|[2008.0,17.0,1484...| 6434.266666666666| 5495.0|
|[2008.0,22.0,1058...| 6625.144444444444| 5995.0|
+--------------------+------------------+-------+
only showing top 10 rows



In [0]:
#display(predicted.limit(1000))
predicted_rf1.limit(1000).show()

+--------------------+------------------+-------+
|            features|        prediction|  label|
+--------------------+------------------+-------+
|[1997.0,17.0,1645...|3879.0555555555557| 1500.0|
|[2002.0,14.0,1712...|10613.944444444445| 8950.0|
|[2003.0,18.0,1482...|3778.5333333333333| 2800.0|
|[2005.0,15.0,2820...|31339.666666666668|24900.0|
|[2006.0,15.0,1282...| 6070.874358974358| 4995.0|
|[2006.0,17.0,1365...|5875.3076923076915| 5589.0|
|[2006.0,20.0,8719...|            7449.1| 5900.0|
|[2008.0,15.0,310....|           48372.8|48999.0|
|[2008.0,17.0,1484...| 6434.266666666666| 5495.0|
|[2008.0,22.0,1058...| 6625.144444444444| 5995.0|
|[2009.0,18.0,2781...| 6309.433333333333| 2300.0|
|[2011.0,13.0,1299...|13357.269999999999|17900.0|
|[2012.0,14.0,9504...|18580.633333333335|19391.0|
|[2012.0,15.0,1668...|           14699.9| 8389.0|
|[2012.0,18.0,7600...|11787.650584795321|12000.0|
|[2012.0,21.0,5665...|14257.787777777778|11000.0|
|[2012.0,25.0,1147...| 8180.689040404041| 9489.0|


In [0]:
rf_evaluator_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse") 
print("RMSE for only RF: %.2f" % rf_evaluator_rmse.evaluate(prediction_rf1))

 
rf_evaluator_r2 = RegressionEvaluator(predictionCol="prediction", \
                labelCol="label",metricName="r2")
print("R2 for only RF: %.2f" % rf_evaluator_r2.evaluate(prediction_rf1))


RMSE for only RF: 6937.48
R2 for only RF: 0.72


In [0]:
assembler_fm = VectorAssembler(inputCols = ["year", "city_fuel_economy", "mileage", "owner_count", "highway_fuel_economy", "engine_displacement", "savings_amount"], outputCol="features").setHandleInvalid("skip")

In [0]:
training_fm = assembler_fm.transform(train).select(col("features"))
#training.show()

In [0]:
fm = FMRegressor(labelCol='label') #,featuresCol="features", numTrees=10, maxDepth=5

In [0]:
# paramGrid_CV_fm = ParamGridBuilder()\
#   .addGrid(fm.factorSize, [4, 7]) \
#   .addGrid(fm.stepSize, [50, 90]) \
#   .addGrid(fm.regParam, [0.0]) \
# .build()


paramGrid_CV_fm = ParamGridBuilder() \
  .addGrid(fm.stepSize, [0.1, 1]) \
  .build()

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
fm_evaluator = RegressionEvaluator(predictionCol="prediction", \
labelCol="label",metricName="r2")

In [0]:
pipeline_fm = Pipeline(stages=[assembler_fm, fm]) 

In [0]:
cv_fm = CrossValidator(estimator=pipeline_fm, evaluator=fm_evaluator, estimatorParamMaps=paramGrid_CV_fm)

In [0]:
model_fm = cv_fm.fit(train) 

In [0]:
prediction_fm = model_fm.transform(test)
predicted_fm = prediction_fm.select("features", "prediction", "label") 
predicted_fm.show(10)

+--------------------+-------------------+-------+
|            features|         prediction|  label|
+--------------------+-------------------+-------+
|[2002.0,14.0,1712...|-298262.64678397554| 8950.0|
|[2003.0,18.0,1482...| -84135.67164692322| 2800.0|
|[2004.0,16.0,1434...|-107169.90338720895| 4889.0|
|[2006.0,17.0,1276...| -63605.10398977945| 4488.0|
|[2007.0,17.0,2449...| -358831.7283427908| 4795.0|
|[2008.0,13.0,846....| 489369.83487343596|34900.0|
|[2008.0,15.0,310....| 443561.60182840435|48999.0|
|[2008.0,22.0,1058...|-22214.505959814876| 5995.0|
|[2011.0,18.0,8840...| -64012.23207307626|12921.0|
|[2011.0,18.0,3699...| 45026.773507431964|18950.0|
+--------------------+-------------------+-------+
only showing top 10 rows



In [0]:
fm_cv_evaluator_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse") 
print("RMSE for FM_CV: %.2f" % fm_cv_evaluator_rmse.evaluate(prediction_fm))

fm_cv_evaluator_r2 = RegressionEvaluator(predictionCol="prediction", \
                labelCol="label",metricName="r2")
print("R2 for FM_CV: %.2f" % fm_cv_evaluator_r2.evaluate(prediction_fm))


RMSE for FM_CV: 95536.98
R2 for FM_CV: -48.92


In [0]:
#display(predicted_fm.limit(1000))

In [0]:
paramGrid_TV_fm = ParamGridBuilder()\
  .addGrid(fm.factorSize, [8]) \
  .addGrid(fm.stepSize, [0.01, 0.1]) \
  .addGrid(fm.regParam, [0.01, 0.1]) \
.build()

In [0]:
fm_tv_evaluator_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse") 
print("RMSE for FM_TV: %.2f" % fm_tv_evaluator_rmse.evaluate(prediction_tv_fm))

fm_tv_evaluator_r2 = RegressionEvaluator(predictionCol="prediction", \
                labelCol="label",metricName="r2")
print("R2 for FM_CV: %.2f" % fm_tv_evaluator_r2.evaluate(prediction_tv_fm))

RMSE for FM_TV: 19841.19
R2 for FM_CV: -1.15


In [0]:
prediction_tv_fm = model_tv_fm.transform(test)
predicted_tv_fm = prediction_tv_fm.select("features", "prediction", "label") 
predicted_tv_fm.show(10)

+--------------------+------------------+-------+
|            features|        prediction|  label|
+--------------------+------------------+-------+
|[2002.0,14.0,1712...| 51432.75495778801| 8950.0|
|[2003.0,18.0,1482...| 7414.746483148912| 2800.0|
|[2004.0,16.0,1434...|11052.810717561719| 4889.0|
|[2006.0,17.0,1276...| 6903.443750543498| 4488.0|
|[2007.0,17.0,2449...| 43637.93900190329| 4795.0|
|[2008.0,13.0,846....|28289.174439077844|34900.0|
|[2008.0,15.0,310....|25074.657019378872|48999.0|
|[2008.0,22.0,1058...| 2827.426917106175| 5995.0|
|[2011.0,18.0,8840...|23677.419623396643|12921.0|
|[2011.0,18.0,3699...|13887.143528103643|18950.0|
+--------------------+------------------+-------+
only showing top 10 rows



In [0]:
fm_tv = TrainValidationSplit(estimator=pipeline_fm, evaluator=fm_evaluator, estimatorParamMaps=paramGrid_TV_fm, trainRatio=0.8)


In [0]:
model_tv_fm = fm_tv.fit(train)

In [0]:
display(predicted_tv_fm.limit(1000))

features,prediction,label
"Map(vectorType -> dense, length -> 7, values -> List(2002.0, 14.0, 171277.0, 8.0, 17.0, 4000.0, 1715.0))",51432.75495778801,8950.0
"Map(vectorType -> dense, length -> 7, values -> List(2003.0, 18.0, 148234.0, 3.0, 25.0, 3000.0, 0.0))",7414.746483148912,2800.0
"Map(vectorType -> dense, length -> 7, values -> List(2004.0, 16.0, 143494.0, 1.0, 23.0, 3500.0, 0.0))",11052.810717561719,4889.0
"Map(vectorType -> dense, length -> 7, values -> List(2006.0, 17.0, 127669.0, 2.0, 22.0, 3000.0, 0.0))",6903.443750543498,4488.0
"Map(vectorType -> dense, length -> 7, values -> List(2007.0, 17.0, 244956.0, 2.0, 24.0, 3500.0, 998.0))",43637.93900190329,4795.0
"Map(vectorType -> dense, length -> 7, values -> List(2008.0, 13.0, 846.0, 2.0, 18.0, 6100.0, 5794.0))",28289.174439077844,34900.0
"Map(vectorType -> dense, length -> 7, values -> List(2008.0, 15.0, 310.0, 2.0, 24.0, 7000.0, 4253.0))",25074.657019378872,48999.0
"Map(vectorType -> dense, length -> 7, values -> List(2008.0, 22.0, 105892.0, 3.0, 30.0, 2400.0, 0.0))",2827.426917106175,5995.0
"Map(vectorType -> dense, length -> 7, values -> List(2011.0, 18.0, 88401.0, 1.0, 26.0, 3500.0, 1308.0))",23677.419623396643,12921.0
"Map(vectorType -> dense, length -> 7, values -> List(2011.0, 18.0, 36996.0, 2.0, 25.0, 3500.0, 1248.0))",13887.143528103643,18950.0


Databricks visualization. Run in Databricks to view.

In [0]:
flightSchema = StructType([
  StructField("year", IntegerType(), False),
  StructField("price", IntegerType(), False),
  StructField("mileage", StringType(), False),
  StructField("owner_count", IntegerType(), False),
  StructField("seller_rating", IntegerType(), False),
  StructField("city_fuel_economy", IntegerType(), False),
  StructField("highway_fuel_economy", IntegerType(), False),
  StructField("daysonmarket", IntegerType(), False),
  StructField("dealer_zip", IntegerType(), False),
  StructField("engine_displacement", IntegerType(), False),
  StructField("listing_id", IntegerType(), False),
  StructField("savings_amount", IntegerType(), False),
  StructField("seller_rating", IntegerType(), False),
  StructField("sp_id", IntegerType(), False),
    
])

In [0]:
(df2.select("year", "city_fuel_economy", col("price").alias("label"), "mileage", "owner_count", "latitude", "highway_fuel_economy", "daysonmarket", "dealer_zip", "engine_displacement", "listing_id", "savings_amount", "seller_rating","sp_id").limit(10)).show()


+----+-----------------+-------+--------+-----------+--------+--------------------+------------+----------+-------------------+----------+--------------+-------------+------+
|year|city_fuel_economy|  label| mileage|owner_count|latitude|highway_fuel_economy|daysonmarket|dealer_zip|engine_displacement|listing_id|savings_amount|seller_rating| sp_id|
+----+-----------------+-------+--------+-----------+--------+--------------------+------------+----------+-------------------+----------+--------------+-------------+------+
|2019|             19.0|55994.0|    14.0|       null|  30.222|                26.0|         716|     32244|               3500| 219569238|             0|            4|371785|
|null|             25.0|   null|    null|       null|    null|                null|          27|     32256|               null|      null|          null|         null|  null|
|null|             12.0|   null|    null|       null|    null|                null|          48|     50126|               nul

In [0]:
df2 = df2.filter(col("price") < 100000)
#display(df2.limit(10))
(df2.limit(10)).show()
df23 = df2.select("year", "city_fuel_economy", ((col("price") > 10000).cast("Double").alias("label")), "mileage", "owner_count", "highway_fuel_economy", "daysonmarket", "dealer_zip", "engine_displacement", "listing_id", "savings_amount", "seller_rating","sp_id")


+-----------------+------------+----+----------+----------+---------------+-----+------------+-----------------+--------------------+------------+----------+--------------------+----------------+-------------------+-----------+--------------------+-----+-------------+----------------+--------------+-------------+----------------+---------+-------------+-------+--------------------+----------+-------------------+-----+------------+------+------+---------+--------+--------+-----------+-------------+----------+---------+--------------------+--------------------+-------------+---------------+--------+----------+-----------+------------------+-------+-------+--------------+-------------+------+--------------------+-----------+--------------------+------------+--------------------+------+--------------------+-----------------------+------------+--------------------+---------+-------+----+
|              vin|back_legroom| bed|bed_height|bed_length|      body_type|cabin|        city|city_fuel_

In [0]:
df23.na.drop(subset=["label", "year", "city_fuel_economy", "mileage","owner_count", "highway_fuel_economy", "daysonmarket", "dealer_zip", "engine_displacement", "listing_id", "savings_amount", "seller_rating","sp_id"])
df23.show()


+----+-----------------+-----+--------+-----------+--------------------+------------+----------+-------------------+----------+--------------+-------------+------+
|year|city_fuel_economy|label| mileage|owner_count|highway_fuel_economy|daysonmarket|dealer_zip|engine_displacement|listing_id|savings_amount|seller_rating| sp_id|
+----+-----------------+-----+--------+-----------+--------------------+------------+----------+-------------------+----------+--------------+-------------+------+
|2019|             19.0|  1.0|    14.0|       null|                26.0|         716|     32244|               3500| 219569238|             0|            4|371785|
|2018|             22.0|  1.0| 19559.0|          1|                28.0|         100|     32246|               2000| 273093755|          2081|            4|283437|
|2006|             20.0|  0.0|119181.0|       null|                28.0|           8|     32216|               2400| 281035985|          2141|            4|289407|
|2020|          

In [0]:
splits = df23.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 2971  Testing Rows: 1318


In [0]:
# the following columns are categorical number such as ID so that it should be Category features
catVect = VectorAssembler(inputCols = ["mileage","year", "owner_count", "city_fuel_economy", "highway_fuel_economy", "daysonmarket", "dealer_zip", "engine_displacement", "listing_id", "savings_amount", "seller_rating","sp_id"], outputCol="catFeatures").setHandleInvalid("skip")
catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), outputCol = "idxCatFeatures").setHandleInvalid("skip")

In [0]:
# number is meaningful so that it should be number features
numVect = VectorAssembler(inputCols = ["mileage"], outputCol="numFeatures")
# number vector is normalized
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")

featVect = VectorAssembler(inputCols=["idxCatFeatures", "normFeatures"], outputCol="features")  #="features1")
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)

# Pipeline process the series of transformation above, which is 7 transformation
pipeline = Pipeline(stages=[catVect, catIdx, numVect, minMax, featVect, lr]) #minMax2, lr])

In [0]:
paramGrid = (ParamGridBuilder() \
             .addGrid(lr.regParam, [0.01, 0.5]) \
             .addGrid(lr.elasticNetParam, [0.0, 0.5]) \
             .addGrid(lr.maxIter, [1, 5]) \
             .build())
#paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.3, 0.1, 0.01]).addGrid(lr.maxIter, [10, 5]).addGrid(lr.threshold, [0.35, 0.30]).build()

In [0]:
#cv_fm = CrossValidator(estimator=pipeline_fm, evaluator=fm_evaluator, estimatorParamMaps=paramGrid_fm)
cv_logr = CrossValidator(estimator=pipeline, evaluator=BinaryClassificationEvaluator(), estimatorParamMaps=paramGrid)
model_Cv_logr = cv_decision.fit(train)

In [0]:
prediction_cv_decision = model_Cv_decision.transform(test)
predicted_cv_decisison = prediction_cv_decision.select("features", "prediction", "probability", "trueLabel")

predicted_cv_decisison.show(100, truncate=False)

+--------------------------------------------------------------------------------------------------------+----------+------------------------------------------+---------+
|features                                                                                                |prediction|probability                               |trueLabel|
+--------------------------------------------------------------------------------------------------------+----------+------------------------------------------+---------+
|[164524.0,1997.0,1.0,17.0,27.0,84.0,50613.0,3800.0,2.74364588E8,0.0,1.0,284048.0,0.5191587384231868]    |0.0       |[0.9922863937205131,0.007713606279486918] |0.0      |
|[148234.0,2003.0,2.0,18.0,25.0,77.0,50613.0,3000.0,2.74961437E8,0.0,1.0,284048.0,0.46775532099525097]   |0.0       |[0.9670723363322165,0.03292766366778355]  |0.0      |
|[105733.0,2004.0,1.0,15.0,21.0,12.0,32216.0,3500.0,2.80686438E8,1458.0,1.0,289407.0,0.33364257427304717]|0.0       |[0.6494344318173253,0.350565

In [0]:
tp = float(predicted_cv_decisison.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted_cv_decisison.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted_cv_decisison.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted_cv_decisison.filter("prediction == 0.0 AND truelabel == 1").count())
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn))],["metric", "value"])

In [0]:
metrics.show() 

+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|             417.0|
|       FP|              25.0|
|       TN|              68.0|
|       FN|               3.0|
|Precision|0.9434389140271493|
|   Recall|0.9928571428571429|
+---------+------------------+



In [0]:
prediction_cv_decision.select("rawPrediction", "probability", "prediction", "trueLabel").show(100, truncate=False)

+----------------------------------------+------------------------------------------+----------+---------+
|rawPrediction                           |probability                               |prediction|trueLabel|
+----------------------------------------+------------------------------------------+----------+---------+
|[4.857025950216553,-4.857025950216553]  |[0.9922863937205131,0.007713606279486918] |0.0       |0.0      |
|[3.3799601521205886,-3.3799601521205886]|[0.9670723363322165,0.03292766366778355]  |0.0       |0.0      |
|[0.6165541198317896,-0.6165541198317896]|[0.6494344318173253,0.3505655681826747]   |0.0       |0.0      |
|[1.8352438407166574,-1.8352438407166574]|[0.8623852332264639,0.13761476677353612]  |0.0       |0.0      |
|[-0.5687089749936831,0.5687089749936831]|[0.36153477546928825,0.6384652245307118]  |1.0       |1.0      |
|[2.90629853128803,-2.90629853128803]    |[0.9481569189758158,0.05184308102418422]  |0.0       |0.0      |
|[-6.463630001837373,6.46363000183737

In [0]:
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc_cv = evaluator.evaluate(prediction_cv_decision)
print("AUC = ", auc_cv)

AUC =  0.9795442908346135


In [0]:

cv = TrainValidationSplit(estimator=pipeline, evaluator=BinaryClassificationEvaluator(), estimatorParamMaps=paramGrid, trainRatio=0.8)

model = cv.fit(train)

In [0]:
prediction = model.transform(test)
predicted = prediction.select("features", "prediction", "probability", "trueLabel")

predicted.show(100, truncate=False)

+--------------------------------------------------------------------------------------------------------+----------+------------------------------------------+---------+
|features                                                                                                |prediction|probability                               |trueLabel|
+--------------------------------------------------------------------------------------------------------+----------+------------------------------------------+---------+
|[164524.0,1997.0,1.0,17.0,27.0,84.0,50613.0,3800.0,2.74364588E8,0.0,1.0,284048.0,0.5191587384231868]    |0.0       |[0.9869464838835124,0.01305351611648764]  |0.0      |
|[148234.0,2003.0,2.0,18.0,25.0,77.0,50613.0,3000.0,2.74961437E8,0.0,1.0,284048.0,0.46775532099525097]   |0.0       |[0.9556442943448961,0.04435570565510394]  |0.0      |
|[105733.0,2004.0,1.0,15.0,21.0,12.0,32216.0,3500.0,2.80686438E8,1458.0,1.0,289407.0,0.33364257427304717]|0.0       |[0.6644182815204104,0.335581

In [0]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn))],["metric", "value"])

In [0]:
metrics.show() 

+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|             415.0|
|       FP|              28.0|
|       TN|              65.0|
|       FN|               5.0|
|Precision|0.9367945823927766|
|   Recall|0.9880952380952381|
+---------+------------------+



In [0]:
prediction.select("rawPrediction", "probability", "prediction", "trueLabel").show(100, truncate=False)

+------------------------------------------+------------------------------------------+----------+---------+
|rawPrediction                             |probability                               |prediction|trueLabel|
+------------------------------------------+------------------------------------------+----------+---------+
|[4.325558285282682,-4.325558285282682]    |[0.9869464838835124,0.01305351611648764]  |0.0       |0.0      |
|[3.070144415665254,-3.070144415665254]    |[0.9556442943448961,0.04435570565510394]  |0.0       |0.0      |
|[0.6830463942228562,-0.6830463942228562]  |[0.6644182815204104,0.3355817184795896]   |0.0       |0.0      |
|[1.532250236613777,-1.532250236613777]    |[0.822335312156212,0.17766468784378797]   |0.0       |0.0      |
|[-0.28256085347209137,0.28256085347209137]|[0.42982606085218644,0.5701739391478136]  |1.0       |1.0      |
|[2.6125102131156837,-2.6125102131156837]  |[0.9316623887535664,0.06833761124643356]  |0.0       |0.0      |
|[-5.30622350347664

In [0]:
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(prediction)
print("AUC = ","{:.2f}".format(auc))

AUC =  0.98
